In [1]:
# Set API Parameters Here
import requests 
import pandas 
import numpy

#Without a valid token and the correct api_url, you will not be able to request any of the files below. '
token = 'This is not public'
api_url = 'This is also not public'

headers={'Authorization': 'Bearer %s' % token}


In [2]:
from geopy.geocoders import Nominatim
from random import randint

In [3]:
user_agent = 'user_me_{}'.format(randint(10000,99999))
geolocator = Nominatim(user_agent=user_agent)

In [4]:
#triprequests
TRqQr = {"query": "{triprequest(input:{next:{fetch:4000}}){id scheduled_ts pickup_ts pickup_address dropoff_ts dropoff_address requested_ts created_by special_assistance organization_id ride_type status estimated_miles fare discount route_id updated_at}}"}
Rq = requests.post(api_url, headers=headers, json=TRqQr)
DtnrRp = Rq.json()
TRq = pandas.DataFrame(DtnrRp["data"]["triprequest"])



In [5]:
TRq.iloc[0:1, :]

,id,scheduled_ts,pickup_ts,pickup_address,dropoff_ts,dropoff_address,requested_ts,created_by,special_assistance,organization_id,ride_type,status,estimated_miles,fare,discount,route_id,updated_at
0,286014,2019-09-14T11:35:00Z,2019-09-14T11:36:46Z,"50 N 3rd St, Columbus, OH 43215, USA",2019-09-14T11:59:36Z,"1356 Cherry Way Drive, Gahanna, OH, USA",2019-09-03T19:59:43Z,589,false,22,Ready By,Completed,11.8,14.0,0.0,39152.0,2019-09-14T11:59:36Z


In [ ]:
#TRq.to_csv("TRq.csv")

In [ ]:
#TRq = pandas.read_csv("TRq.csv")

In [ ]:
#TRq.iloc[0:5, :]

In [7]:
LOUnRID =  TRq["route_id"][TRq["route_id"].notnull()][1:].unique()

In [8]:
len(LOUnRID)

1148

In [9]:
LOUnRID

array([39154., 39159., 39158., ..., 68206., 68241., 68264.])

In [19]:
#route
FRtQr = {"query": "{route(input:{next:{fetch:1000}}){id name driver_id start_ts end_ts vehicle_id ambulatory_riders handicapped_riders total_riders status updated_at}}"}
FRtRq = requests.post(api_url, headers=headers, json=FRtQr)
DtnrRp = FRtRq.json()

In [20]:
Rt = pandas.DataFrame(DtnrRp["data"]["route"])

In [23]:
Rt.iloc[0:1, :]

,id,name,driver_id,start_ts,end_ts,vehicle_id,ambulatory_riders,handicapped_riders,total_riders,status,updated_at
0,37593,R-Q22,1965,2019-08-22T10:38:56Z,2019-08-22T13:53:50Z,2.0,2,0,2,Completed,2019-09-16T15:18:23Z


In [ ]:
FRtQr = {"query": "{route(input:{id: 39152 next:{fetch:10}}){id name driver_id start_ts end_ts vehicle_id ambulatory_riders handicapped_riders total_riders status updated_at}}"}
FRtRq = requests.post(api_url, headers=headers, json=FRtQr)
DtnrRp = FRtRq.json()

for RID in LOUnRID:
    IRID = int(RID)
    TprrRtQr = {"query": "{route(input:{id: " + str(IRID) +  " next:{fetch:10}}){id name driver_id start_ts end_ts vehicle_id  ambulatory_riders handicapped_riders total_riders status updated_at}}"}
    TprrRtRq = requests.post(api_url, headers=headers, json=TprrRtQr)
    TprrDtnrRp = TprrRtRq.json()
    DtnrRp["data"]["route"] = DtnrRp["data"]["route"] + TprrDtnrRp["data"]["route"]
    
    

In [13]:
Rt = pandas.DataFrame(DtnrRp["data"]["route"])


In [24]:
len(Rt["id"])

1000

In [ ]:
#Rt.to_csv("Rt.csv")

In [ ]:
#Rt = pandas.read_csv("Rt.csv")

In [ ]:
#Rt.iloc[0:1, :]

In [25]:
#At most 1000 Random organization rows. 
QrOgnzt = {"query": "{organization(input:{next:{fetch:1000}}){organization_id type pricing_mode}}"}
Rq = requests.post(api_url, headers=headers, json=QrOgnzt)
DtnrRp = Rq.json()
Ognzt= pandas.DataFrame(DtnrRp["data"]["organization"])
Ognzt.to_csv("Ognzt.csv")

In [26]:
#Ognzt = pandas.read_csv("Ognzt.csv")

In [27]:
FStDtQr = {"query": "{safetydetail(input:{vehicle_id: 51 driver_id :3320  route_id: 39152  next:{fetch:10}}){vehicle_id driver_id route_id safety_score  safety_score_rank total_distance_driven harsh_turning_events harsh_braking_events harsh_acceleration_events updated_at}}"}
FStDtRq = requests.post(api_url, headers=headers, json=FStDtQr)
DtnrRp = FStDtRq.json()



In [ ]:
LOVhcIDDvIARID = list(zip(Rt["vehicle_id"][Rt["vehicle_id"].notnull()], Rt["driver_id"][Rt["driver_id"].notnull()], Rt["id"][Rt["id"].notnull()]))
for VhcID,DvID,RID, in LOVhcIDDvIARID[1:]:
    TprrStDtQr = {"query": "{safetydetail(input:{vehicle_id: " + str(VhcID) +  " driver_id : " + str(DvID) + " route_id: " + str(RID) + " next:{fetch:10}}){vehicle_id driver_id route_id safety_score  safety_score_rank total_distance_driven harsh_turning_events harsh_braking_events harsh_acceleration_events updated_at}}"}
    TprrStDtRq = requests.post(api_url, headers=headers, json=TprrStDtQr)
    TprrDtnrRp = TprrStDtRq.json()
    DtnrRp["data"]["safetydetail"] = DtnrRp["data"]["safetydetail"] + TprrDtnrRp["data"]["safetydetail"]
    
    


In [ ]:
StDt = pandas.DataFrame(DtnrRp["data"]["safetydetail"])

In [ ]:
StDt.to_csv("StDt_FFTsTRq.csv")

In [ ]:
StDt = pandas.read_csv("StDt_FOTsTRq.csv")

In [ ]:
len(StDt["vehicle_id"])

In [ ]:
StDt.iloc[0:5, :]

In [29]:
#At most 1000 random vehicle rows. 
VhcQr = {"query": "{vehicle(input:{next:{fetch:1000}}){id vin name make model year capacity handicapped updated_at}}"}
VhcRq = requests.post(api_url, headers=headers, json=VhcQr)
DtnrRp = VhcRq.json()

In [30]:
Vhc = pandas.DataFrame(DtnrRp["data"]["vehicle"])

In [31]:
Vhc.iloc[0:1, :]

,id,vin,name,make,model,year,capacity,handicapped,updated_at
0,1,2F***************,01-H-0004,Ford,Flex,2018,6,0,2019-10-30T09:53:29Z


In [ ]:
len(Vhc["id"])

In [ ]:
#Vhc.to_csv("Vhc.csv")

In [ ]:
FVhcQr = {"query": "{vehicle(input:{id: 51 next:{fetch:1}}){id vin name make model year capacity handicapped updated_at}}"}
FVhcRq = requests.post(api_url, headers=headers, json=FVhcQr)
DtnrRp = FVhcRq.json()


In [ ]:
LOUnVhcID = Rt["vehicle_id"][Rt["vehicle_id"].notnull()].unique()

In [ ]:
len(LOUnVhcID)

In [ ]:
LOUnVhcID = Rt["vehicle_id"][Rt["vehicle_id"].notnull()].unique()
for VhcID in LOUnVhcID[1:]:
    TprrVhcQr = {"query": "{vehicle(input:{id: " + str(VhcID) + " next:{fetch:1}}){id vin name make model year capacity handicapped updated_at}}"}
    TprrVhcRq = requests.post(api_url, headers=headers, json=TprrVhcQr)
    TprrDtnrRp = TprrVhcRq.json()
    DtnrRp["data"]["vehicle"] = DtnrRp["data"]["vehicle"] + TprrDtnrRp["data"]["vehicle"]
    

In [ ]:
Vhc = pandas.DataFrame(DtnrRp["data"]["vehicle"])

In [ ]:
Vhc.to_csv("VhcFFTsTRq.csv")

In [ ]:
Vhc = pandas.read_csv("Vhc_FOTsTRq.csv")

In [ ]:
len(Vhc)

In [ ]:
LOUnRtID = Rt["id"][Rt["id"].notnull()].unique()

In [35]:
FHEvQr = {"query": "{harshevent(input:{next:{fetch:100}}){id route_id event_datetime event_type address distracted incident_url created_datetime updated_at}}"}
FHEvRq = requests.post(api_url, headers=headers, json=FHEvQr)
DtnrRp = FHEvRq.json()

In [36]:
HEv = pandas.DataFrame(DtnrRp["data"]["harshevent"])

In [37]:
HEv.iloc[0:1, :]

,id,route_id,event_datetime,event_type,address,distracted,incident_url,created_datetime,updated_at
0,112,39668,2019-09-09T11:55:21Z,Harsh Braking,Olentangy Freeway; Moody Jackson Parkway (SR 3...,false,https://cloud.samsara.com/groups/18722/fleet/r...,2019-09-10T09:00:25Z,2019-09-10T09:00:25Z


In [ ]:
len(HEv)

In [ ]:
HEv.to_csv("HEv.csv")

In [ ]:
HEv = pandas.read_csv("HEv.csv")

In [ ]:
HEv.iloc[1:10, :]

In [ ]:
#At most 1000 Random harshevent rows. 
HEvQr = {"query": "{harshevent(input:{next:{fetch:1000}}){id route_id event_datetime event_type address distracted incident_url created_datetime updated_at}}"}
HEvRq = requests.post(api_url, headers=headers, json=HEvQr)
DtnrRp = HEvRq.json()
HEv = pandas.DataFrame(DtnrRp["data"]["harshevent"])
HEv.to_csv("HEv.csv")

In [ ]:
HEv = pandas.read_csv("HEv.csv")

In [ ]:
len(HEv["id"])

In [41]:
#vehicle location
FVhcLctQr = {"query": "{vehiclelocation(input:{next:{fetch:100}}){vehicle_id event_timestamp lat lng speed_mph}}"}
FVhcLctRq = requests.post(api_url, headers=headers, json=FVhcLctQr)
DtnrRp = FVhcLctRq.json()

In [ ]:
LOUnVhcID = Vhc["id"].unique()
for VhcID in LOUnVhcID[1:]:
    TprrVhcLctQr = {"query": "{vehiclelocation(input:{vehicle_id: " + str(VhcID) + " next:{fetch:100}}){vehicle_id event_timestamp lat lng speed_mph}}"}
    TprrVhcLctRq = requests.post(api_url, headers=headers, json=TprrVhcLctQr)
    TprrDtnrRp = TprrVhcLctRq.json()
    DtnrRp["data"]["vehiclelocation"] = DtnrRp["data"]["vehiclelocation"] + TprrDtnrRp["data"]["vehiclelocation"]

In [43]:
VhcLct = pandas.DataFrame(DtnrRp["data"]["vehiclelocation"])

In [44]:
VhcLct.iloc[0:1, :]

,vehicle_id,event_timestamp,lat,lng,speed_mph
0,1,2018-11-01T00:32:58.897Z,40.062763,-83.000116,0.0


In [ ]:
VhcLct.to_csv("VhcLctFFTsTRq.csv")

In [ ]:
VhcLct = pandas.read_csv("VhcLct.csv")

In [ ]:
VhcLct

In [45]:
#At most 1000 Random vehiclediagnostic rows. 
VhcDntQr = {"query": "{vehiclediagnostic(input:{next:{fetch:1000}}){vehicle_id event_timestamp check_engine_light dtc_shortcode dtc_id dtc_id2 dtc_description}}"}
VhcDntRq = requests.post(api_url, headers=headers, json=VhcDntQr)
DtnrRp = VhcDntRq.json()


In [46]:
VhcDnt = pandas.DataFrame(DtnrRp["data"]["vehiclediagnostic"])

In [47]:
VhcDnt.iloc[0:1, :]

,vehicle_id,event_timestamp,check_engine_light,dtc_shortcode,dtc_id,dtc_id2,dtc_description
0,1,2018-12-06T00:54:13.573Z,False,None,None,None,None


In [ ]:
VhcDnt.to_csv("VhcDnt.csv")

In [ ]:
LOUnVhcID

In [ ]:
TprrVhcDntQr = {"query": "{vehiclediagnostic(input:{vehicle_id : " + str(900) + " next:{fetch:1000}}){vehicle_id event_timestamp check_engine_light dtc_shortcode dtc_id dtc_id2 dtc_description}}"}
TprrVhcDntRq = requests.post(api_url, headers=headers, json=TprrVhcDntQr)
TprrDtnrRp = TprrVhcDntRq.json()
TprrDtnrRp

In [ ]:
FVhcDntQr = {"query": "{vehiclediagnostic(input:{vehicle_id: 51 next:{fetch:100}}){vehicle_id event_timestamp check_engine_light dtc_shortcode dtc_id dtc_id2 dtc_description}}"}
FVhcDntRq = requests.post(api_url, headers=headers, json=FVhcDntQr)
DtnrRp = FVhcDntRq.json()
for VhcID in LOUnVhcID[1:]:
    TprrVhcDntQr = {"query": "{vehiclediagnostic(input:{vehicle_id : " + str(VhcID) + " next:{fetch:100}}){vehicle_id event_timestamp check_engine_light dtc_shortcode dtc_id dtc_id2 dtc_description}}"}
    TprrVhcDntRq = requests.post(api_url, headers=headers, json=TprrVhcDntQr)
    TprrDtnrRp = TprrVhcDntRq.json()
    DtnrRp["data"]["vehiclediagnostic"] = DtnrRp["data"]["vehiclediagnostic"] + TprrDtnrRp["data"]["vehiclediagnostic"]
    

In [ ]:
VhcDnt = pandas.DataFrame(DtnrRp["data"]["vehiclediagnostic"])
    

In [ ]:
VhcDnt.to_csv("VhcDnt_FFTsTRq.csv")

In [ ]:
VhcDnt